### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
21,42026327,90579,79085,51530,0,0,1118,5,2024-12-12 22:00,2024-12-12 22:00,0,0,4857,4,1
22,42026256,90578,79085,3,0,0,1118,6,2024-12-13 21:59,2024-12-13 22:00,0,0,5464,7,1
23,42025856,79085,12378,51530,0,0,1119,0,2024-12-14 22:00,2024-12-14 22:00,1,4,1271,1,1
24,42026939,47293,79085,3,0,0,1119,1,2024-12-15 19:29,2024-12-15 19:31,0,0,9649,2,1
25,42029022,79085,47939,51530,0,0,1119,2,2024-12-16 22:00,2024-12-16 22:00,0,0,3244,4,1
26,42029823,79085,9502,51530,0,0,1119,3,2024-12-17 22:00,2024-12-17 22:00,0,5,2506,6,1
27,42030133,104653,79085,3,0,0,1119,4,2024-12-18 18:27,2024-12-18 18:27,5,3,1942,2,1
28,42276527,47939,79085,51530,0,0,1119,5,2024-12-19 18:00,2024-12-19 18:00,1,2,2963,2,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

8

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,90579,27744,35789,0,10,1,0,0,5 - 3,11,10,11,42026327
1,79085,42548,34503,2,2,0,0,0,3-5-2aSL,8,9,11,42026327
0,90578,31927,35146,0,6,1,1,0,FechBoia,7,7,6,42026256
1,79085,38022,34803,0,9,3,3,0,4-3-3a,6,5,5,42026256
0,79085,45764,33237,0,3,0,0,0,3-5-2aVG,8,9,9,42025856
1,12378,24722,37249,1,17,3,2,0,ANOMALO XW54,9,11,9,42025856
0,47293,26667,33841,0,6,2,1,0,GDT defensive,7,8,6,42026939
1,79085,43286,36112,1,3,2,1,0,5-4-1a,5,6,6,42026939
0,79085,41395,33318,1,4,4,0,0,3-4-3aQJ,8,9,10,42029022
1,47939,28963,37040,0,10,0,0,0,Bralers,12,9,9,42029022


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1450

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,122720,0,0,0,0,0,0,0,12,1349
1,23755,0,0,0,0,0,0,0,11,1349
2,47841,0,0,0,0,0,0,0,10,1349
3,6295,0,0,0,0,0,0,0,9,1349
4,124139,0,0,0,0,0,0,0,8,1349
5,78671,0,0,0,0,0,0,0,7,1349
6,79085,0,0,0,0,0,0,0,6,1349
7,79866,0,0,0,0,0,0,0,5,1349
8,122169,0,0,0,0,0,0,0,4,1349
9,10365,0,0,0,0,0,0,0,3,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()